In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

In [2]:
pip install openpyxl


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


**PHASE 1**

**DATA INGESTION**

In [3]:
RetailData_df = pd.read_excel("RetailData.xlsx")

In [4]:
RetailData_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
RetailData_df.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [6]:
RetailData_df.shape

(541909, 8)

In [7]:
RetailData_df.describe

<bound method NDFrame.describe of        InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

               InvoiceDate  UnitPrice  Cu

In [8]:
RetailData_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [9]:
# Converting the invoiceDate column to datetime format

RetailData_df['InvoiceDate'] = pd.to_datetime(RetailData_df['InvoiceDate'])

**PHASE 2**

**DATA STORAGE(SQL layer)**

In [ ]:
# database connection setup
database_url = os.getenv("DATABASE_URL")

In [ ]:
# loading environment variables from .env file
load_dotenv()

True

In [ ]:
# Creating a database engine
engine = create_engine(database_url)

In [ ]:
# Loading data into the PostgreSQL database
RetailData_df.to_sql('RetailData', engine, if_exists='append', index=False, schema='retail_data')

909

**PHASE 3**

**DATA PREPARATION (CLEANING & TRANSFORMATION)**

In [15]:
import pandas as pd  # data reading and manipulation
from sqlalchemy import create_engine # database connection and querying
import os  # avoid hardcoding file paths
from dotenv import load_dotenv  # load environment variables from .env file

In [16]:
load_dotenv()

True

In [18]:
r_query = '''SELECT *
FROM retail_data."RetailData";'''

In [20]:
RetailData_df = pd.read_sql(r_query, con=engine)

In [23]:
RetailData_df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [ ]:
# 1. Quantity validation
# Any quantity less than 1 is invalid
invalid_qty = RetailData_df[RetailData_df['Quantity'] < 1]
print("Invalid Quantity transactions:")
print(invalid_qty.head())

Invalid Quantity transactions:
    InvoiceNo StockCode                       Description  Quantity  \
142   C536379         D                          Discount        -1   
155   C536383    35004C   SET OF 3 COLOURED  FLYING DUCKS        -1   
236   C536391     22556    PLASTERS IN TIN CIRCUS PARADE        -12   
237   C536391     21984  PACK OF 12 PINK PAISLEY TISSUES        -24   
238   C536391     21983  PACK OF 12 BLUE PAISLEY TISSUES        -24   

            InvoiceDate  UnitPrice  CustomerID         Country  
142 2010-12-01 09:41:00      27.50     14527.0  United Kingdom  
155 2010-12-01 09:49:00       4.65     15311.0  United Kingdom  
236 2010-12-01 10:24:00       1.65     17548.0  United Kingdom  
237 2010-12-01 10:24:00       0.29     17548.0  United Kingdom  
238 2010-12-01 10:24:00       0.29     17548.0  United Kingdom  


In [26]:
# Filtering out invalid transactions
valid_qty = RetailData_df[RetailData_df['Quantity'] >= 1].copy()

print("\nRows BEFORE filtering:", len(RetailData_df))
print("Rows AFTER filtering:", len(valid_qty))



Rows BEFORE filtering: 541909
Rows AFTER filtering: 531285


In [ ]:
# 